
**Created by:**

__Viktor Varga__

<br>

<img src="https://docs.google.com/uc?export=download&id=1q8cQBQKSLqS3PirWEmtQyObewgHOVISl" style="display:inline-block" width='40%'>
<hr>

# Python tutorial - Python vs. Numpy, futási idők összehasonlítása

A Numpy a Python egyik legfontosabb csomagja: gyors numerikus számítások elvégzésére alkalmas. A Python nyelv alapértelmezett fordítóprogramja/értelmezője a CPython. Ez az implementáció összehasonlítva más nyelvekkel, nagyságrendekkel lassabb programokat eredményez. Ennek főbb okai a Python-t kényelmessé tevő dinamikus típusozásban, automatikus memóriakezelésben és abban keresendő, hogy a CPython szinte teljes mértékben értelmezőként (interpreterként) működik, a kód optimalizálására így nem igazán van lehetőség.

A sebességkülönbség akkor ütközik ki igazán, ha sok Python utasítást kell végrehajtani, például ha egy mátrixszorzást ciklusokkal és skalárok összeszorzásával valósítunk meg. A Numpy éppen ezért olyan függvényeket tartalmaz, melyek képesek ugyanazt a műveletet vektorosan, sokszor egymás után végrehajtani, anélkül, hogy Python ciklust kellene használnunk. A Numpy függvények hatékony nyelveken (pl. C, Fortran) írt kódokat hívnak meg, így a naiv Python megvalósításnál sokkal nagyobb hatékonyság érhető el Numpy használatával.

Kezdetnek, hasonlítsuk össze ugyanannak a feladatnak a megoldását Python-ban Numpy használata nélkül, illetve Numpy használatával!
A feladat: két nagy méretű mátrix véletlen generálása és összeszorzása.

In [ ]:
# Pure Python implementation (matrices: lists of lists)
import random
import time

def multiply_random_matrices(matrix_dim):
  matrix1 = [[random.random() for col_idx in range(matrix_dim)] for row_idx in range(matrix_dim)]
  matrix2 = [[random.random() for col_idx in range(matrix_dim)] for row_idx in range(matrix_dim)]
  matrix_out = [[0. for col_idx in range(matrix_dim)] for row_idx in range(matrix_dim)]

  for row1_idx in range(matrix_dim):
    for col2_idx in range(matrix_dim):
      for item_idx in range(matrix_dim):
        matrix_out[row1_idx][col2_idx] += matrix1[row1_idx][item_idx] * matrix2[item_idx][col2_idx]

  return matrix_out

# We will measure execution time
time_start = time.time()
size = 400
multiply_random_matrices(size)
time_end = time.time()

print("Multiplying ", size, "x", size, " random matrices in pure Python takes",\
          time_end - time_start, "seconds.")


Multiplying  400 x 400  random matrices in pure Python takes 14.79543662071228 seconds.


Két 400 x 400-as méretű véletlen számokat tartalmazó mátrix legenerálása és összeszorzása kb. 13 másodpercig tartott a Colab gépein. Most nézzük ugyanezt Numpy-ban.

In [ ]:
# Numpy implementation (matrices: ndarrays)
import numpy as np
import time

def multiply_random_matrices_npy(matrix_dim):
  matrix1 = np.random.rand(matrix_dim, matrix_dim)
  matrix2 = np.random.rand(matrix_dim, matrix_dim)

  return np.matmul(matrix1, matrix2)

# We will measure execution time
time_start = time.time()
size = 400
multiply_random_matrices_npy(size)
time_end = time.time()

print("Multiplying ", size, "x", size, " random matrices in Numpy takes",\
          time_end - time_start, "seconds.")

Multiplying  400 x 400  random matrices in Numpy takes 0.019975900650024414 seconds.


A Numpy megfelelő függvényével ez 0.015 másodpercig tartott, azaz körülbelül 800-szor gyorsabb ez az implementáció.

Most nézzünk meg egy másik példát: generáljunk véletlen számokkal feltöltött kis méretű mátrixokat és páronként szorozzunk össze őket.

In [ ]:
# Pure Python implementation (matrices: lists of lists)

# We will measure execution time
time_start = time.time()
size = 3
n_matrix_pairs = 1000000
for mat_idx in range(n_matrix_pairs):
  multiply_random_matrices(size)
time_end = time.time()

print("Multiplying ", n_matrix_pairs, "pairs of", size, "x", size,\
        " random matrices in pure Python takes", time_end - time_start, "seconds.")

Multiplying  1000000 pairs of 3 x 3  random matrices in pure Python takes 13.798646926879883 seconds.


1 millió 3x3-as mátrixpár véletlen generálása és páronkénti összeszorzása Numpy használata nélkül körülbelül 13 másodpercig tart a Colab gépein. Nézzük ugyanezt Numpy-al:

In [ ]:
# Numpy implementation with np.matmul and loop

# We will measure execution time
time_start = time.time()
size = 3
n_matrix_pairs = 1000000
for mat_idx in range(n_matrix_pairs):
  multiply_random_matrices_npy(size)
time_end = time.time()

print("Multiplying ", n_matrix_pairs, "pairs of", size, "x", size,\
        " random matrices in Numpy takes", time_end - time_start, "seconds.")

Multiplying  1000000 pairs of 3 x 3  random matrices in Numpy takes 3.9823391437530518 seconds.


Ugyanez Numpy-al 4 másodpercig tart. Korántsem akkora a gyorsulás, mint az első esetben. Mi okozhatja ezt?

A fenti kódban minden egyes 3x3-as mátrixpár generálásához és összeszorzásához összesen három Numpy függvényhívást teszünk meg. Ciklusban, 1 millió pár előállításához és összeszorzásához 3 millió Numpy hívást végzünk el, ami jelentős időt vesz igénybe. Továbbá, ahelyett, hogy egyszerre foglalnánk le az összes mátrixot a műveletvégzéshez, 3 millió alkalommal allokálunk apró területeket a memóriában, ami ugyancsak sokáig tart.

Nézzük meg hogyan lehetne ezt hatékonyabban, ciklus nélkül megoldani:

In [ ]:
# Numpy implementation, multiple matrices at once

def multiply_random_matrices_npy_atonce(matrix_dim, n_matrix_pairs):
  matrix1 = np.random.rand(n_matrix_pairs, matrix_dim, matrix_dim)
  matrix2 = np.random.rand(n_matrix_pairs, matrix_dim, matrix_dim)

  return np.matmul(matrix1, matrix2)  # matmul handles 3D arrays as stacks of matrices

# We will measure execution time
time_start = time.time()
size = 3
n_matrix_pairs = 1000000
multiply_random_matrices_npy_atonce(size, n_matrix_pairs)
time_end = time.time()

print("Multiplying ", size, "x", size, " random matrices in Numpy without loop takes",\
          time_end - time_start, "seconds.")

Multiplying  3 x 3  random matrices in Numpy without loop takes 0.4543569087982178 seconds.


A Numpy legtöbb művelete támogatja egyszerre sok azonos művelet elvégzését különböző adatokon. Az `np.matmul()` mátrixszorzás művelet esetében még extra paraméter megadása sem szükséges, automatikusan felismeri, hogy a bemeneti mátrixok nem 2, hanem 3 dimenziósak, melyeket mátrixok tömbjeként kezel. Python ciklus nélkül 0.4 másodperc alatt fut le a művelet, tízszer gyorsabban, mint Numpy-ban Python ciklusokkal és harmincszor gyorsabban, mint tisztán Pythonban.